# Analyze types of MPA

In [17]:
import geojson
import pandas as pd
import numpy as np

In [20]:
# Load geojson file
path = '../assets/California_Marine_Protected_Areas_[ds582].geojson'
with open(path) as f:
    gj = geojson.load(f)

# Extract Type, FULLNAME, OBJECTID
dict_MPA = {'OBJECTID':[], 'Type':[], 'FULLNAME':[]}
for i in range(len(gj['features'])):
    for key in dict_MPA.keys():
        dict_MPA[key].append(gj['features'][i]['properties'][key])
        
# Create dataframe from dict_MPA
df_MPA = pd.DataFrame.from_dict(dict_MPA)

In [25]:
pd.unique(df_MPA['Type'])

array(['SMCA', 'Special Closure', 'SMR', 'SMRMA', 'SMP', 'SMCA (No-Take)',
       'FMR', 'FMCA'], dtype=object)

In [26]:
df_MPA['Type'].value_counts()

SMCA               61
SMR                49
Special Closure    14
SMCA (No-Take)     10
FMR                 8
SMP                 7
SMRMA               5
FMCA                1
Name: Type, dtype: int64